In [1]:
import requests

ticker = "AAPL"
url = f"https://api.pushshift.io/reddit/search/comment/?q={ticker}&size=3"
response = requests.get(url, timeout=10)
print(response.status_code)
print(response.json())


403
{'detail': 'Not authenticated'}


In [1]:
import requests
from bs4 import BeautifulSoup

def fetch_valueresearch_search(company: str):
    url = f"https://www.valueresearchonline.com/search/?q={company}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    results = []
    for article in soup.select(".search-results .article-item"):
        title = article.select_one("h3").get_text(strip=True)
        summary = article.select_one("p").get_text(strip=True)
        link = "https://www.valueresearchonline.com" + article.select_one("a")["href"]
        results.append((title, summary, link))
    return results


In [5]:
company = "TCS"
results = fetch_valueresearch_search(company)
results

[]

In [6]:
import feedparser


company_name = "Netweb Technologies"
ticker = "NETWEB"

feed = feedparser.parse("https://www.valueresearchonline.com/rss/stocks.xml")

items = []
for entry in feed.entries[:50]:
    title = entry.get("title", "").lower()
    summary = entry.get("summary", "").lower()

    if ticker.lower() in title or ticker.lower() in summary or \
       company_name.lower() in title or company_name.lower() in summary:
        items.append(entry)

items

[]

In [9]:
import requests
from datetime import datetime
import logging



def get_trendlyne_stock_id(ticker: str) -> str | None:
    """
    Resolve Trendlyne numeric ID for a stock ticker.
    """
    try:
        url = "https://trendlyne.com/member/api/ac_snames/stock/"
        headers = {
            "User-Agent": "Mozilla/5.0",
            "x-requested-with": "XMLHttpRequest",
        }
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
        data = resp.json()  # returns a list of stock dicts
        for item in data:
            if item.get("symbol") == ticker.upper():
                return str(item.get("id"))
        #logger.warning(f"Ticker {ticker} not found in Trendlyne stock list.")
    except Exception as e:
        #logger.error(f"Error fetching Trendlyne stock ID: {e}")
        pass
    return None

def fetch_trendlyne_news(ticker: str) -> list[dict]:
    """
    Fetch latest news for a given stock from Trendlyne.
    """
    news_items = []
    stock_id = get_trendlyne_stock_id(ticker)
    if not stock_id:
        return news_items

    news_url = f"https://trendlyne.com/equity/api/getNewsByStockId/{stock_id}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "x-requested-with": "XMLHttpRequest",
    }

    try:
        resp = requests.post(news_url, headers=headers, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        for entry in data.get("data", []):
            published = datetime.strptime(entry.get("published_at"), "%Y-%m-%d %H:%M:%S")
            news_items.append(
                {
                    "title": entry.get("title"),
                    "link": entry.get("url"),
                    "date": published,
                    "source": entry.get("source_name"),
                }
            )
        #logger.info(f"Fetched {len(news_items)} news items for {ticker}.")
    except Exception as e:
        #logger.error(f"Error fetching Trendlyne news: {e}")
        pass

    return news_items



In [11]:
ticker = "INFY"
 
news = fetch_trendlyne_news(ticker)
for n in news[:5]:  # first 5 items
    print(n.date, n.title)
news


[]

In [ ]:
# Get-ChildItem -Recurse -Directory "C:\Users\Admin\.virtualenvs\equity_forecasting_env" | Select-Object -ExpandProperty FullName



In [ ]:
import pandas as pd

df = pd.read_parquet("C:/Users/Admin/equity_forecasting/datalake/data/cache/features/AAPL.parquet")
print(df.columns.tolist())
print(df.tail())

In [5]:
import pandas as pd
df = pd.read_parquet("C:/Users/Admin/equity_forecasting/datalake/runs/RUN_20251224_153655/dataset/Y_pooled.parquet")
print(df.columns.tolist())
print(df.head())

['target']
      target
0  32.272499
1  32.340000
2  32.134998
3  31.602501
4  31.650000
